In [4]:
import pybullet
import gym
import numpy as np
import matplotlib.pyplot as plt
import pybullet_data
import math

In [ ]:
#!git clone https://github.com/ROBOTIS-GIT/turtlebot3.git

In [92]:
physicsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setGravity(0,0,-9.8)
planeID = pybullet.loadURDF('plane.urdf')
chess_grid = pybullet.loadTexture('checker_huge.gif')
pybullet.changeVisualShape(planeID, -1, textureUniqueId=chess_grid)


#common_props = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/common_properties.xacro')

turtle1 = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.25,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
turtle2 = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
turtle3 = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,0.75,0], pybullet.getQuaternionFromEuler([0,0,0]))
turtle4 = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [0.75,-0.25,0], pybullet.getQuaternionFromEuler([0,0,0]))
turtle5 = pybullet.loadURDF('turtlebot3/turtlebot3_description/urdf/turtlebot3_waffle_pi.urdf.xacro', [1.25,0.75,0], pybullet.getQuaternionFromEuler([0,0,0]))

#robot_texture = pybullet.loadTexture('turtlebot3/turtlebot3_description/rviz/waffle_pi.rviz')
#robot_texture = pybullet.loadTexture('duckCM.png')


pybullet.changeVisualShape(turtle1, -1, rgbaColor=[0,0,1,1])
pybullet.changeVisualShape(turtle2, -1, rgbaColor=[0,0,1,1])
pybullet.changeVisualShape(turtle3, -1, rgbaColor=[0,0,1,1])
pybullet.changeVisualShape(turtle4, -1, rgbaColor=[0,0,1,1])
pybullet.changeVisualShape(turtle5, -1, rgbaColor=[0,0,1,1])











#pybullet.changeVisualShape(turtle1, -1, textureUniqueId=robot_texture)
#pybullet.changeVisualShape(turtle2, -1, textureUniqueId=robot_texture)
#pybullet.changeVisualShape(turtle3, -1, textureUniqueId=robot_texture)
#pybullet.changeVisualShape(turtle4, -1, textureUniqueId=robot_texture)
#pybullet.changeVisualShape(turtle5, -1, textureUniqueId=robot_texture)

viewMatrix = pybullet.computeViewMatrix(
            cameraEyePosition=[0.5,0,3],
            cameraTargetPosition=[0.5,0,0],
            cameraUpVector=[0,1,0])

projectionMatrix = pybullet.computeProjectionMatrixFOV(
            fov=45.0,
            aspect=1.0,
            nearVal=0.1,
            farVal=3.1)

width, height, rgbImg, depthImg, segImg = pybullet.getCameraImage(width = 224, height = 224, viewMatrix=viewMatrix, projectionMatrix = projectionMatrix)


pybullet.setRealTimeSimulation(1)



In [91]:
pybullet.disconnect()